In [2]:
import numpy as np
from Trip import Trip
from Point import Point
from Trips import Trips
import math
import matplotlib.pyplot as plt
import gurobipy as gb


# CC

In [42]:
#  customized clustering procedure
def cc_procedure(J,R,D):
    # n is number of realocation moves J={1, ..., n}
    # m is number of realocators R={1, ..., m}
    n=len(J)
    m=len(R)

    #initialize a new Trip
    new_trip=Trip([],[],0)

    p=np.floor(n/m)/(np.floor(n/m)+1)
    n_hat=0
    # controllare
    if np.random.uniform(0,1)<=p:
        n_hat=np.random.randint(np.ceil((m+1)/2),m)
    else:
        n_hat=np.random.randint(1,np.ceil((m+1)/2)) #TODO sistemare che metta il secondo incluso

    j=J[np.random.randint(0,len(J))] # len(J) = |J|
    # update
    new_trip.J.append(j) #J'={j} # TODO sistemare si può mettere un metodo dentro Trip
    J.remove(j) # J=J\{j}   P= (u_j^c,v_j^c)
    
    while len(new_trip.J)<n_hat and len(J)!=0:
        p=[]
        for j in J:
            p_j=0.0
            for q in J:
                distance_j=new_trip.distance_from_cluster_center(j)
                distance_q=new_trip.distance_from_cluster_center(q)
                p_j+=distance_j/distance_q if distance_q!=0 else 0
            p.append(p_j**-1 if p_j!=0 else 0.0)
        j=np.random.choice(J, 1, p)[0] # sample from J
        new_trip.J.append(j) #J'={j} # TODO sistemare si può mettere un metodo dentro Trip
        J.remove(j) # J=J\{j}

    l=1
    pi=[]

    # line 7
    probability=[]
    for selected_pi in D:
        p_pi=0.0
        for q in D:
            distance_pi=new_trip.distance_from_cluster_center(selected_pi)
            sum_distance_dropoff=new_trip.distance_from_cluster_center(q)
            p_pi+=distance_pi/sum_distance_dropoff if sum_distance_dropoff!=0 else 0
        probability.append(p_pi**-1 if p_pi!=0 else 0.0)

    pi_l=np.random.choice(D, 1, probability)[0]
    new_trip.pi.append(pi_l) # pi={pi_l} #TODO  sistemare con un metodo
    D.remove(pi_l) # D=D\{pi_l}

    #calculate total duration C of taxi trip pi according to relocation J'
    C=new_trip.duration()

    while len(D)!=0:
        #determine drop-off point q in D which decreases C most
        if(q != None):
            l=l+1
            pi_l=q
            D.remove(q)
        else:
            D=[]

# SA


In [ ]:
# FIX IT
# TODO error when n=10, m=1 because n_realocation_moves becomes 0 so all trips have 0 reallocation moves except the last one which has all reallocation moves
def sa_approach(n,m,ks,kr,kn,T_start,c,J,D):
    # ks -> number of restarts
    # kr -> number of rehats
    # T_start -> initial temperature
    # c -> cooling parameter
    # By varying these parameters and caching not only the best solution found so far, but also the temporary best solution, i.e., before reheat and restart
    # we obtain a pool of taxi trips by splitting the cached solutions, i.e., taxi tour, into the single trips

    for i in range(ks):
        # generate random feasible solution
        # set initial current state sc
        n_trips=int(np.ceil(n/(np.ceil((m+1)/2))))
        trips=[]
        for i in range(n_trips):
            k=np.random.randint(1,m+1) # number of drop-off points
            pi=np.random.choice(D, k,replace=False) #These drop-off points are randomly chosen

            # the relocation moves are randomly spread between the different trips.

            n_realocation_moves=int(len(J)/n_trips) # TODO we can improve using the normal distribution to select the right dimension

            J_prime=np.random.choice(J, n_realocation_moves,replace=False)
            if i==n_trips-1:
                J_prime=J.copy()
            for j in J_prime:
                J.remove(j)
            trip=Trip(J_prime,pi,k)
            trips.append(trip)
            # Finally, for each trip and its assigned relocation moves, we execute each relocation from the closest drop-off point increasing the trip duration least.

            # if I understood well, it means we have to take the closest drop off point to the relocation moves to select in the time duration

        trips_star=trips.copy() # optimal solution

        for j in range(kr):
            T=T_start
            not_improved=0
            new_trips=trips.copy()
            while not_improved<kn:
                not_improved+=1
                # Sn= neighbour of current state Sc

                selected_indexes_trips=np.random.choice(range(len(new_trips)), 2,replace=False)
                print(selected_indexes_trips)
                trip1=new_trips[selected_indexes_trips[0]].copy()
                trip2=new_trips[selected_indexes_trips[1]].copy()

                # select an action
                n_actions=np.random.randint(0,5)
                match n_actions:
                    case 0:
                        # Swap two relocation moves between two taxi trips
                        Trips.swap_random_realocation_moves(trip1,trip2)
                    case 1:
                        # Move a relocation move to another taxi trip
                        Trips.move_random_realocation_moves(trip1,trip2)

                    case 2:
                        # Remove a drop-off point from a taxi trip
                        Trips.remove_random_dropoff(trip1)

                    case 3:
                        # Swap two drop-off points between two taxi trips
                        Trips.swap_random_dropoff(trip1,trip2)

                    case 4:
                        # Add a drop-off point to a taxi trip
                        pi=np.random.choice(D, 1)[0]
                        Trips.add_drop_off(trip1,pi)

                new_trips[selected_indexes_trips[0]]=trip1
                new_trips[selected_indexes_trips[1]]=trip2

                # TODO sistemare sbagliato

                z_trips=Trips.get_total_duration(trips)
                z_new_trips=Trips.get_total_duration(new_trips)

                # TODO sistemare sbagliato
                if(z_new_trips<z_trips):
                    # sc=sn update current
                    trips=new_trips
                    z_trips=z_new_trips
                    not_improved=0
                    z_trips_star=Trips.get_total_duration(trips_star)
                    if(z_trips<z_trips_star):
                        trips_star=trips
                else:
                    #with probability exp() assign sc=sn
                    exp_lambda= (z_new_trips-z_trips)/T
                    exponential_sample=np.random.exponential(1/exp_lambda) # TODO sistemare calcolo probabilità
                    trips=new_trips if exponential_sample<1/exp_lambda else trips # sc=sn if probability else sc
                T=T*c


# SAM

In [ ]:
def sam_matheuristic(trips, J, D):
    m=1
    # I-> trips
    I=range(len(trips))

    # preparate the variables
    C=[]
    for i in I:
        C.append(trips[i].get_pure_taxi_trip())

    delta=np.zeros((len(trips),len(J)))


    for i in I:
        for j in J: # TODO sitemare sbagliato diff
            diff=10-C[i]
            delta[i,j]=max(0,diff)





    sam_mip = gb.Model()
    sam_mip.modelSense = gb.GRB.MAXIMIZE #declare mimization
    X = sam_mip.addVars( [(i,j) for i in I for j in range(len(J))], vtype=gb.GRB.BINARY) #this way of declare vars does not allow to work with matrix multiplication (X is a tuple dict)
    Y = sam_mip.addVars( [i for i in I], vtype=gb.GRB.BINARY)
    #Xi,j==1
    for j in range(len(J)):
        sam_mip.addConstr( gb.quicksum(X[i,j] for i in I) == 1 )
    # X(i,j)<=Y(i)*m
    for i in I:
        sam_mip.addConstr( gb.quicksum(X[i,j] for j in range(len(J))) <= Y[i] * m)
    # TODO sistemare
    b=[]


    for j in range(len(J)):
        for i in I: # b[i]>=X[i,j]*delta[i,j]
            sam_mip.addConstr( b[i] >= X[i,j] * delta[i,j])

    sam_mip.setObjective(gb.quicksum( Y[i]*C[i]+b[i]  for i in I))

    sam_mip.optimize()

    print( "\n", type(X), X, "\n")
    print("\nSolution")

    for i in I:
      for j in range(len(J)):
        if X[i,j].x == 1:  #to access the variable value
          print("Machine ", i, " is assigned to task ", j)

# SM

In [ ]:
def sm_matheuristic(trips,J,D):
    I=range(len(trips))
    # to construct theta
    # TODO Controllare
    theta=np.zeros((len(trips),len(J)))
    for i in I:
        for j in range(len(J)):
            theta[i,j]=1 if any(obj.u == j.u and obj.v==j.v for obj in trips[i].J) else 0
    C=[]
    sm_mip = gb.Model()
    sm_mip.modelSense = gb.GRB.MAXIMIZE #declare mimization
    Y = sm_mip.addVars( [i for i in I], vtype=gb.GRB.BINARY)
    for j in range(len(J)):
        sm_mip.addConstr( gb.quicksum(Y[i]*theta[i,j] for i in I) >= 1 )

    sm_mip.setObjective(gb.quicksum( Y[i]*C[i]  for i in I))
    sm_mip.optimize()
    print( "\n", type(Y), Y, "\n")
    print("\nSolution")

    for i in I:
        print(f'Y[{i}]= {Y[i]}')

# Local search

In [ ]:
def local_search(trips,J,D,kn):
    initial_solution=trips.copy()
    notImproved=0
    while notImproved<kn:
        notImproved+=1
        # select current neighborhood with equal probability
        # generate new solution sn from sc for the current neighborhood
        new_solution=0 # TODO sistemare
        z_trips=Trips.get_total_duration(initial_solution)
        z_new_trips=Trips.get_total_duration(new_solution)

        if(z_new_trips<z_trips):
            # sc=sn update current
            initial_solution=new_solution
            z_trips=z_new_trips
            not_improved=0
    return initial_solution

In [ ]:
def trptr_problem():
